In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import re

### Load Data

In [2]:
# datetime of each financial report
with open('../10-Q/fp_url_htm/company_dt_dic_final.pkl','rb') as f:
    company_dt_dic = pickle.load(f)
    
# htm or txt url of each financial report
with open('../10-Q/fp_url_htm/company_fp_dic_final.pkl','rb') as f:
    company_fp_dic = pickle.load(f)

# htm or txt url of each financial report
with open('../10-Q/fp_url_htm/company_pop_dic_final.pkl','rb') as f:
    company_pop_dic = pickle.load(f)

    
with open('../10-K/fp_url_htm/company_fp_dic_final.pkl','rb') as f:
    company_fp_dic_10k = pickle.load(f)

with open('../10-K/fp_url_htm/company_dt_dic_final.pkl','rb') as f:
    company_dt_dic_10k = pickle.load(f)

with open('../10-K/fp_url_htm/company_pop_dic_final.pkl','rb') as f:
    company_pop_dic_10k = pickle.load(f)
          

In [112]:
for k,v in company_pop_dic.items():
    if(type(v) != str):
        try:
            company_pop_dic[k] = np.unique(v)
            company_dt_dic[k] = np.unique(company_dt_dic[k])
            
#             assert len(v) == np.unique(v).shape[0]
        except:
            print(k)

In [52]:
sorted(company_dt_dic["A"]+company_dt_dic_10k["A"])

[datetime.datetime(2000, 3, 15, 0, 0),
 datetime.datetime(2000, 6, 12, 0, 0),
 datetime.datetime(2000, 9, 1, 0, 0),
 datetime.datetime(2001, 1, 17, 0, 0),
 datetime.datetime(2001, 3, 19, 0, 0),
 datetime.datetime(2001, 6, 14, 0, 0),
 datetime.datetime(2001, 9, 14, 0, 0),
 datetime.datetime(2002, 1, 22, 0, 0),
 datetime.datetime(2002, 3, 6, 0, 0),
 datetime.datetime(2002, 6, 5, 0, 0),
 datetime.datetime(2002, 9, 13, 0, 0),
 datetime.datetime(2002, 12, 20, 0, 0),
 datetime.datetime(2003, 3, 12, 0, 0),
 datetime.datetime(2003, 6, 4, 0, 0),
 datetime.datetime(2003, 9, 9, 0, 0),
 datetime.datetime(2003, 12, 22, 0, 0),
 datetime.datetime(2004, 3, 4, 0, 0),
 datetime.datetime(2004, 6, 3, 0, 0),
 datetime.datetime(2004, 9, 2, 0, 0),
 datetime.datetime(2004, 12, 21, 0, 0),
 datetime.datetime(2005, 3, 10, 0, 0),
 datetime.datetime(2005, 6, 6, 0, 0),
 datetime.datetime(2005, 9, 7, 0, 0),
 datetime.datetime(2006, 1, 17, 0, 0),
 datetime.datetime(2006, 3, 9, 0, 0),
 datetime.datetime(2006, 6, 6, 0,

### Get all company name 

In [3]:
dealing_data=pd.read_csv("dealing_data.csv",index_col=0,parse_dates=True)
all_company=dealing_data.columns

In [54]:
np.sum(all_company=='AAPL')

1

In [217]:
dealing_data

,A,AA,AAA,AAAB,AAABB,AAAP,AAAU,AAB,AABA,AABC,...,ZU,ZUMZ,ZUO,ZVUE,ZVX,ZVXI,ZX,ZYME,ZYNE,ZZ
1993-01-04,0.024390,0.019197,-0.038462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-05,0.000000,0.001712,-0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-06,0.011905,-0.006838,0.130435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-07,-0.035294,-0.027539,0.153846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-08,0.012195,-0.001770,0.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-11,-0.012048,0.005319,0.187500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-12,0.000000,-0.015873,-0.052632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-13,-0.024390,-0.016129,-0.055556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-14,0.037500,0.000000,-0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-15,0.012048,0.014572,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#rest_company=list(set(all_company_data.index.get_level_values(0)))

NameError: name 'all_company_data' is not defined

In [ ]:
#with open('dealing_list.txt', 'w') as f:
#    for item in rest_company:
#        f.write("%s\n" % item)

In [ ]:
#all_company_data.to_csv("all_company_data.csv")

### Get the return
如果Ｑ相同則取最新的，所有日期都要在2018年底之前

In [18]:
SP_return=pd.read_csv("SP_return.csv",index_col=0,parse_dates=True)
SP_price=(SP_return+1).cumprod()["sprtrn"]

In [8]:
Q10=pd.DataFrame(company_fp_dic[all_company[0]][::-1],index=company_dt_dic[all_company[0]][::-1])
Q10[1]=pd.Series(["Q"]*len(Q10), index=Q10.index)

In [9]:
K10=pd.DataFrame(company_fp_dic_10k[all_company[0]][::-1],index=company_dt_dic_10k[all_company[0]][::-1])
K10[1]=pd.Series(["K"]*len(K10), index=K10.index)

In [80]:
temp=pd.concat([Q10_pop,K10_pop]).sort_index()

In [81]:
temp.columns=[["url","QK"]]

In [82]:
np.array([i for i in range(len(all_company))])[all_company == 'AAPL']

array([44])

In [38]:
# some test
# i=44 # AAPL
i = 243
cumprod_return=[]
datetime_record=[]

# if(type(company_fp_dic[all_company[i]]) =='str' ):
#     print(company_fp_dic[all_company[i]])
    
# else:
Q10=pd.DataFrame(company_fp_dic[all_company[i]][::-1],index=company_dt_dic[all_company[i]][::-1])
Q10[1]=pd.Series(["Q"]*len(Q10), index=Q10.index)

Q10_pop=pd.DataFrame(company_fp_dic[all_company[i]][::-1],index=company_pop_dic[all_company[i]][::-1])
Q10_pop[1]=pd.Series(["Q"]*len(Q10_pop), index=Q10_pop.index)

K10=pd.DataFrame(company_fp_dic_10k[all_company[i]][::-1],index=company_dt_dic_10k[all_company[i]][::-1])
K10[1]=pd.Series(["K"]*len(K10), index=K10.index)

K10_pop=pd.DataFrame(company_fp_dic_10k[all_company[i]][::-1],index=company_pop_dic_10k[all_company[i]][::-1])
K10_pop[1]=pd.Series(["K"]*len(K10_pop), index=K10_pop.index)

temp=pd.concat([Q10,K10]).sort_index()
temp.columns=[["url","QK"]]
datetime=list(temp.index)


bad_company=[]

for j in range(len(datetime)-1):
    if(datetime[j+1].year < 2019):
        diff_day = ((datetime[j+1]+dt.timedelta(-7))-(datetime[j]+dt.timedelta(7))).days
#         print(diff_day)

        if(diff_day < 0):

            if(temp.loc[datetime[j+1]]['QK'].values == 'Q' and temp.loc[datetime[j]]['QK'].values  == 'Q'):
                print('Type1')
#                 continue

            elif(temp.loc[datetime[j+1]]['QK'].values  == 'K' and temp.loc[datetime[j]]['QK'].values  == 'Q'):
                print('Type2')
                diff_day_q = (datetime[j]-datetime[j-1]).days
#                 datetime[j] = datetime[j-1]+dt.timedelta(int(diff_day_q)/2)

                start_time = datetime[j-1]+dt.timedelta(int(diff_day_q)/2) + dt.timedelta(7)
                end_time = datetime[j+1]+dt.timedelta(-7)

            elif(temp.loc[datetime[j+1]]['QK'].values  == 'Q' and temp.loc[datetime[j]]['QK'].values  == 'K'):
                print('Type3')
                diff_day_q = (datetime[j+2]-datetime[j+1]).days
#                 datetime[j+1] = datetime[j+2]-dt.timedelta(int(diff_day_q/2))

                start_time = datetime[j]+dt.timedelta(7)
                end_time = datetime[j+2]-dt.timedelta(int(diff_day_q/2))+dt.timedelta(-7)
            else:
                start_time = datetime[j]+dt.timedelta(7)
                end_time = datetime[j+1]+dt.timedelta(-7)
        else:
            start_time = datetime[j]+dt.timedelta(7)
            end_time = datetime[j+1]+dt.timedelta(-7)

#         print(datetime[j],diff_day)

        if (datetime[j+1]-datetime[j]):#.days<120:
#             cumprod_return.append((dealing_data[all_company[i]].loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)].dropna().replace("C",0)+1).cumprod()[-1])
#             datetime_record.append(datetime[j])
#             sp_return_period=SP_price.loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)][-1]/SP_price.loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)][0]
            cumprod_return.append((dealing_data[all_company[i]].loc[start_time:end_time].dropna().replace("C",0)+1).cumprod()[-1])
            datetime_record.append(datetime[j])
            sp_return_period=SP_price.loc[start_time:end_time][-1]/SP_price.loc[start_time:end_time][0]

            if cumprod_return[-1]<sp_return_period-0.2:
                bad_company.append(True)
            else:
                bad_company.append(False)
    #         except:
    #             continue

url=temp["url"].loc[datetime_record].values.reshape(-1).tolist()
QK=temp["QK"].loc[datetime_record].values.reshape(-1).tolist()
arrays = [[all_company[i]]*len(cumprod_return),datetime_record]
tuples = list(zip(*arrays))
index=pd.MultiIndex.from_tuples(tuples, names=['company', 'date'])
single_company_data=pd.DataFrame({'return':cumprod_return,'bad':bad_company,'url':url,'QK':QK},index=index)#.dropna()
single_company_data

return    bad  \
company date                          
ACR     1997-08-14  0.962265  False   
        1997-11-14  1.124998  False   
        1998-03-31  0.960563  False   
        1998-05-15  0.840391  False   
        1998-08-14  0.949219  False   
        1998-11-05  0.832571   True   
        1999-03-29  1.156116  False   
        1999-05-17  0.711744   True   
        1999-08-16  0.426470   True   
        1999-11-15  0.992648  False   
        2000-03-30  0.937500  False   
        2000-05-15  0.816326   True   
        2000-08-14  0.899999  False   
        2000-11-14  0.947620  False   
        2001-03-30  1.013333  False   
        2001-05-15  1.043714  False   
        2001-08-14  0.913042  False   
        2001-11-14  0.671296   True   
        2002-04-01  0.907692  False   
        2002-05-15  0.821053  False   
        2002-08-14  0.948355  False   
        2002-11-14  0.922222  False   
        2003-03-06  0.988888  False   
        2003-05-09  1.189742  False   
        2003-08-08  1.508850  False   
        2003-11-07  1.438235  False   
        2004-03-09  0.953758  False   
        2004-05-07  1.158321  False   
        2004-08-11  1.365316  False   
        2004-11-05  1.555548  False   
        2005-03-04  1.008708  False   
        2005-06-10  0.895005  False   
        2005-08-05  1.282550  False   
        2005-11-07  1.276025  False   
        2006-02-27  0.934785  False   

                                                                  url QK  
company date                                                              
ACR     1997-08-14  /Archives/edgar/data/787784/0000950144-97-0091...  Q  
        1997-11-14  /Archives/edgar/data/787784/0000950144-97-0122...  Q  
        1998-03-31  /Archives/edgar/data/787784/0000950144-98-0039...  K  
        1998-05-15  /Archives/edgar/data/787784/0000950144-98-0065...  Q  
        1998-08-14  /Archives/edgar/data/787784/0000950144-98-0099...  Q  
        1998-11-05  /Archives/edgar/data/787784/0000950144-98-0120...  Q  
        1999-03-29  /Archives/edgar/data/787784/0000950144-99-0034...  K  
        1999-05-17  /Archives/edgar/data/787784/0000950144-99-0062...  Q  
        1999-08-16  /Archives/edgar/data/787784/000095014499010432...  Q  
        1999-11-15  /Archives/edgar/data/787784/000095014499013258...  Q  
        2000-03-30  /Archives/edgar/data/787784/000095014400004269...  K  
        2000-05-15  /Archives/edgar/data/787784/000095014400006883...  Q  
        2000-08-14  /Archives/edgar/data/787784/000095014400010385...  Q  
        2000-11-14  /Archives/edgar/data/787784/000095014400013846...  Q  
        2001-03-30  /Archives/edgar/data/787784/000095014401004461...  K  
        2001-05-15  /Archives/edgar/data/787784/000095014401502493...  Q  
        2001-08-14  /Archives/edgar/data/787784/000095014401505906...  Q  
        2001-11-14  /Archives/edgar/data/787784/000095014401509224...  Q  
        2002-04-01  /Archives/edgar/data/787784/000095014402003182...  K  
        2002-05-15  /Archives/edgar/data/787784/000095014402005603...  Q  
        2002-08-14  /Archives/edgar/data/787784/000095014402008893...  Q  
        2002-11-14  /Archives/edgar/data/787784/000095014402011936...  Q  
        2003-03-06  /Archives/edgar/data/787784/000095014403002752...  K  
        2003-05-09  /Archives/edgar/data/787784/000095014403006371...  Q  
        2003-08-08  /Archives/edgar/data/787784/000115752303003867...  Q  
        2003-11-07  /Archives/edgar/data/787784/000115752303006425...  Q  
        2004-03-09  /Archives/edgar/data/787784/000115752304002165...  K  
        2004-05-07  /Archives/edgar/data/787784/000115752304004540...  Q  
        2004-08-11  /Archives/edgar/data/787784/000115752304007600...  Q  
        2004-11-05  /Archives/edgar/data/787784/000115752304010409...  Q  
        2005-03-04  /Archives/edgar/data/787784/000115752305002217...  K  
        2005-06-10  /Archives/edgar/data/787784/000115752305005412...  Q  
        2005-08-05  /Archive

In [83]:
all_company_data=pd.DataFrame()
for i in range(len(all_company)):  
    # i = 243
    if(all_company[i] not in company_list):
        try:
            cumprod_return=[]
            datetime_record=[]

            Q10=pd.DataFrame(company_fp_dic[all_company[i]][::-1],index=company_dt_dic[all_company[i]][::-1])
            Q10[1]=pd.Series(["Q"]*len(Q10), index=Q10.index)

            Q10_pop=pd.DataFrame(company_fp_dic[all_company[i]][::-1],index=company_pop_dic[all_company[i]][::-1])
            Q10_pop[1]=pd.Series(["Q"]*len(Q10_pop), index=Q10_pop.index)

            K10=pd.DataFrame(company_fp_dic_10k[all_company[i]][::-1],index=company_dt_dic_10k[all_company[i]][::-1])
            K10[1]=pd.Series(["K"]*len(K10), index=K10.index)

            K10_pop=pd.DataFrame(company_fp_dic_10k[all_company[i]][::-1],index=company_pop_dic_10k[all_company[i]][::-1])
            K10_pop[1]=pd.Series(["K"]*len(K10_pop), index=K10_pop.index)

            temp=pd.concat([Q10,K10]).sort_index()
            temp.columns=[["url","QK"]]
            datetime=list(temp.index)

            datetime_record_start = []
            datetime_record_end = []
            bad_company=[]

            for j in range(len(datetime)-1):
                if(datetime[j+1].year < 2019):
                    diff_day = ((datetime[j+1]+dt.timedelta(-7))-(datetime[j]+dt.timedelta(7))).days
            #         print(diff_day)

                    if(diff_day < 0):

                        if(temp.loc[datetime[j+1]]['QK'].values == 'Q' and temp.loc[datetime[j]]['QK'].values  == 'Q'):
            #                 print('Type1')
                            continue

                        elif(temp.loc[datetime[j+1]]['QK'].values  == 'K' and temp.loc[datetime[j]]['QK'].values  == 'Q'):
#                             print('Type2')
                            diff_day_q = (datetime[j]-datetime[j-1]).days
            #                 datetime[j] = datetime[j-1]+dt.timedelta(int(diff_day_q)/2)

                            start_time = datetime[j-1]+dt.timedelta(int(diff_day_q)/2) + dt.timedelta(7)
                            end_time = datetime[j+1]+dt.timedelta(-7)

                        elif(temp.loc[datetime[j+1]]['QK'].values  == 'Q' and temp.loc[datetime[j]]['QK'].values  == 'K'):
#                             print('Type3')
                            diff_day_q = (datetime[j+2]-datetime[j+1]).days
            #                 datetime[j+1] = datetime[j+2]-dt.timedelta(int(diff_day_q/2))

                            start_time = datetime[j]+dt.timedelta(7)
                            end_time = datetime[j+2]-dt.timedelta(int(diff_day_q/2))+dt.timedelta(-7)
                        else:
                            start_time = datetime[j]+dt.timedelta(7)
                            end_time = datetime[j+1]+dt.timedelta(-7)
                    else:
                        start_time = datetime[j]+dt.timedelta(7)
                        end_time = datetime[j+1]+dt.timedelta(-7)

            #         print(datetime[j],diff_day)

                    if (datetime[j+1]-datetime[j]).days<150:
                        try:
            #             cumprod_return.append((dealing_data[all_company[i]].loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)].dropna().replace("C",0)+1).cumprod()[-1])
            #             datetime_record.append(datetime[j])
            #             sp_return_period=SP_price.loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)][-1]/SP_price.loc[datetime[j]+dt.timedelta(7):datetime[j+1]+dt.timedelta(-7)][0]
                            cumprod_return.append((dealing_data[all_company[i]].loc[start_time:end_time].dropna().replace("C",0)+1).cumprod()[-1])
                            datetime_record.append(datetime[j])
                            sp_return_period=SP_price.loc[start_time:end_time][-1]/SP_price.loc[start_time:end_time][0]
                            datetime_record_start.append(start_time+np.timedelta64(-7,'D'))
                            datetime_record_end.append(end_time+np.timedelta64(7,'D'))
                            
                            if cumprod_return[-1]<sp_return_period-0.2:
                                bad_company.append(True)
                            else:
                                bad_company.append(False)
                        except:
                            continue

            url=temp["url"].loc[datetime_record].values.reshape(-1).tolist()
            QK=temp["QK"].loc[datetime_record].values.reshape(-1).tolist()
            arrays = [[all_company[i]]*len(cumprod_return),datetime_record]
            tuples = list(zip(*arrays))
            index=pd.MultiIndex.from_tuples(tuples, names=['company', 'date'])
            single_company_data=pd.DataFrame({'adjust_starttime':datetime_record_start,'adjust_endtime':datetime_record_end,'return':cumprod_return,'bad':bad_company,'url':url,'QK':QK},index=index)#.dropna()
            all_company_data=pd.concat([all_company_data,single_company_data])
        except:
            continue

In [84]:
all_company_data

adjust_starttime adjust_endtime    return  bad  \
company date                                                        
AAPL    1994-12-13       1994-12-13     1994-12-30  0.993610  0.0   
        1994-12-30       1994-12-30     1995-03-31  0.973724  0.0   
        1995-03-31       1995-03-31     1995-06-30  1.330258  0.0   
        1995-12-19       1995-12-19     1996-02-13  0.918606  0.0   
        1995-12-29       1995-12-29     1996-03-29  0.803963  1.0   
        1996-03-29       1996-03-29     1996-06-28  0.937823  0.0   
        1996-12-19       1996-12-19     1997-02-11  0.664865  1.0   
        1996-12-27       1996-12-27     1997-03-28  0.791664  1.0   
        1997-03-28       1997-03-28     1997-06-27  0.824501  1.0   
        1997-12-05       1997-12-05     1997-12-26  0.939914  0.0   
        1997-12-26       1997-12-26     1998-03-27  2.009521  0.0   
        1998-03-27       1998-03-27     1998-06-26  0.990845  0.0   
        1998-12-26       1998-12-26     1999-03-27  0.818319  1.0   
        1999-03-27       1999-03-27     1999-06-26  1.306763  0.0   
        1999-12-22       1999-12-22     2000-02-16  1.147042  0.0   
        2000-01-01       2000-01-01     2000-04-01  1.393849  0.0   
        2000-04-01       2000-04-01     2000-07-01  0.784632  0.0   
        2000-12-14       2000-12-14     2000-12-30  1.043479  0.0   
        2000-12-30       2000-12-30     2001-03-31  1.404582  0.0   
        2001-03-31       2001-03-31     2001-06-30  1.081108  0.0   
        2001-12-29       2001-12-29     2002-03-30  1.016886  0.0   
        2002-03-30       2002-03-30     2002-06-29  0.681083  0.0   
        2002-12-19       2002-12-19     2003-02-12  1.006267  0.0   
        2002-12-28       2002-12-28     2003-03-29  1.006712  0.0   
        2003-03-29       2003-03-29     2003-06-28  1.332411  0.0   
        2003-12-19       2003-12-19     2004-02-11  1.067618  0.0   
        2003-12-27       2003-12-27     2004-03-27  1.215230  0.0   
        2004-03-27       2004-03-27     2004-06-26  1.196726  0.0   
        2004-12-03       2004-12-03     2004-12-25  1.015627  0.0   
        2004-12-25       2004-12-25     2005-03-26  1.334160  0.0   
...                             ...            ...       ...  ...   
SMRT    2009-04-16       2009-04-16     2009-05-02  0.967742  0.0   
        2009-05-02       2009-05-02     2009-08-01  2.429423  0.0   
        2009-08-01       2009-08-01     2009-10-31  0.842016  1.0   
        2010-04-15       2010-04-15     2010-05-01  1.010870  0.0   
        2010-05-01       2010-05-01     2010-07-31  0.916377  0.0   
        2010-07-31       2010-07-31     2010-10-30  1.276388  0.0   
        2011-04-01       2011-04-01     2011-04-30  1.026445  0.0   
        2011-04-30       2011-04-30     2011-07-30  1.023935  0.0   
        2011-07-30       2011-07-30     2011-10-29  0.834614  1.0   
        2012-04-12       2012-04-12     2012-04-28  0.986971  0.0   
        2012-04-28       2012-04-28     2012-07-28  1.311879  0.0   
        2012-07-28       2012-07-28     2012-10-27  0.992424  0.0   
        2013-05-03       2013-05-03     2013-06-19  1.226310  0.0   
        2013-05-04       2013-05-04     2013-08-03  1.285801  0.0   
        2013-08-03       2013-08-03     2013-11-02  1.169851  0.0   
        2014-04-15       2014-04-15     2014-05-03  0.987577  0.0   
        2014-05-03       2014-05-03     2014-08-02  1.042841  0.0   
        2014-08-02       2014-08-02     2014-11-01  0.952416  0.0   
        2015-04-02       2015-04-02     2015-05-02  1.004899  0.0   
        2015-05-02       2015-05-02     2015-08-01  0.843497  0.0   
        2015-08-01       2015-08-01     2015-10-31  0.868653  0.0   
        2016-04-11       2016-04-11     2016-04-30  1.045391  0.0   
        2016-04-30       2016-04-30     2016-07-30  1.249064  0.0   
        2016-07-30       2016-07-30     2016-10-29  0.738731  1.0   
        2017-04-13       2017-04-13     2017-04-29  1.008032  0.0   
        2017-04-2

In [4]:
company_origin = pd.read_csv("all_company_data_ver2.csv")

In [13]:
company_list = np.unique(company_origin['company'].values)

In [91]:
all_company_data.to_csv('extra_all_company_data.csv')

In [92]:
extra_all_company_data = pd.read_csv('extra_all_company_data.csv')

In [87]:
extra_all_company_data['url'].values

array(['/Archives/edgar/data/320193/0000320193-94-000016.txt',
       '/Archives/edgar/data/320193/0000320193-95-000003.txt',
       '/Archives/edgar/data/320193/0000320193-95-000008.txt', ...,
       '/Archives/edgar/data/884940/000119312518152760/d474980d10k.htm',
       '/Archives/edgar/data/884940/000119312518186895/d587137d10q.htm',
       '/Archives/edgar/data/884940/000119312518264999/d592664d10q.htm'],
      dtype=object)

In [108]:
extra_all_company_data['adjust_starttime'] = [i.split(' ')[0] for i in extra_all_company_data['adjust_starttime'].values]
extra_all_company_data['adjust_endtime'] = [i.split(' ')[0] for i in extra_all_company_data['adjust_endtime'].values]

In [95]:
company_origin['adjust_starttime'] = company_origin['date']

In [102]:
np.append(company_origin['date'].values[1:],[np.nan]).shape

(282246,)

In [103]:
company_origin['adjust_endtime'] =  np.append(company_origin['date'].values[1:],np.nan)

In [109]:
all_company_data = pd.concat([company_origin,extra_all_company_data],sort='True')

In [110]:
all_company_data = all_company_data[['company','date','adjust_starttime','adjust_endtime','return','bad','url', 'QK']]


In [111]:
all_company_data

,company,date,adjust_starttime,adjust_endtime,return,bad,url,QK
0,A,2000-03-15,2000-03-15,2000-06-12,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q
1,A,2000-06-12,2000-06-12,2000-09-01,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q
2,A,2000-09-01,2000-09-01,2001-01-17,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q
3,A,2001-01-17,2001-01-17,2001-03-19,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K
4,A,2001-03-19,2001-03-19,2001-06-14,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q
5,A,2001-06-14,2001-06-14,2001-09-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q
6,A,2001-09-14,2001-09-14,2002-01-22,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q
7,A,2002-01-22,2002-01-22,2002-03-06,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K
8,A,2002-03-06,2002-03-06,2002-06-05,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q
9,A,2002-06-05,2002-06-05,2002-09-13,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...,Q


In [112]:
all_company_data.to_csv("all_company_data_ver3.csv",index=False)

In [302]:
pd.read_csv("all_company_data_ver3.csv")

,company,date,adjust_datetime,return,bad,url,QK
0,A,2000-03-15,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q
1,A,2000-06-12,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q
2,A,2000-09-01,2000-09-01,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q
3,A,2001-01-17,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K
4,A,2001-03-19,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q
5,A,2001-06-14,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q
6,A,2001-09-14,2001-09-14,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q
7,A,2002-01-22,2002-01-22,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K
8,A,2002-03-06,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q
9,A,2002-06-05,2002-06-05,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...,Q


In [296]:
'AAPL' in all_company_data['company'].values

True

In [200]:
np.unique(xxx['company'].values).shape[0]

7493

In [375]:
"""
store_url=pd.DataFrame(index=all_company_data.index,columns=["url"])
for i in range(all_company_data.shape[0]):
    index_number=pd.Index(company_dt_dic[all_company_data.index.get_level_values(0)[i]]).get_loc(all_company_data.index.get_level_values(1)[i])
    store_url.iloc[i]=company_fp_dic[all_company_data.index.get_level_values(0)[i]][index_number]
all_company_data=pd.concat([all_company_data,store_url],axis=1)
"""

In [395]:
#all_company_data.to_csv("all_company_data.csv")

In [9]:
bad_date=all_company_data[all_company_data["bad"]==1]
#bad_date.to_csv("bad_date.csv")
bad_date

,company,date,return,bad,url
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...
5,A,2002-06-05,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...
8,A,2004-03-04,0.805133,1.0,/Archives/edgar/data/1090872/00011931250403472...
49,AAC,2018-08-03,0.589634,1.0,/Archives/edgar/data/1606180/00015645901801918...
62,AACC,2010-04-30,0.629009,1.0,/Archives/edgar/data/1264707/00011931251010077...
65,AACC,2011-07-29,0.672450,1.0,/Archives/edgar/data/1264707/00011931251120158...
70,AACE,1997-10-30,0.820945,1.0,/Archives/edgar/data/849116/0000930661-97-0024...
72,AACE,1998-11-04,0.878890,1.0,/Archives/edgar/data/849116/0000849116-98-0000...
77,AACE,2001-02-09,0.728711,1.0,/Archives/edgar/data/849116/000084911601000002...
94,AAI,2001-08-02,0.563954,1.0,/Archives/edgar/data/948846/000093176301501206...


In [10]:
good_date = all_company_data[all_company_data.iloc[:,3]==0]
good_date.head()

,company,date,return,bad,url
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...
2,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...
3,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...
4,A,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...
6,A,2003-03-12,1.192572,0.0,/Archives/edgar/data/1090872/00008916180300121...


In [11]:
good_ticker=[]
with open('./100ticker.txt') as f:
    for line in f.readlines():
        good_ticker.append(line.replace('\n',''))

In [12]:
good_date_df = pd.DataFrame(columns=good_date.columns)

In [13]:
for tic in good_ticker:
    temp = good_date[good_date.iloc[:,0]==tic]
    good_date_df = pd.concat([good_date_df,temp])

In [14]:
good_date_df.head()

,company,date,return,bad,url
339,AAPL,1995-02-09,0.951541,0.0,/Archives/edgar/data/320193/0000320193-95-0000...
340,AAPL,1995-05-15,1.037996,0.0,/Archives/edgar/data/320193/0000320193-95-0000...
341,AAPL,1996-02-12,0.931820,0.0,/Archives/edgar/data/320193/0000320193-96-0000...
343,AAPL,1997-02-10,1.042143,0.0,/Archives/edgar/data/320193/0000320193-97-0000...
344,AAPL,1997-05-12,1.144925,0.0,/Archives/edgar/data/320193/0000320193-97-0000...


In [84]:
"""
測試用code
input_data=good_date_df.iloc[15:20,:]
output=input_data.copy()
output["text"]="NA"
else_deal=[]
for i in range(input_data.shape[0]):
    try:
        print("start")
        index_now=input_data.index[i]
        url="https://www.sec.gov"+input_data["url"].iloc[i]
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup_txt = soup.get_text()
        soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ')
        soup_txt=soup_txt.split(" ")
        while '' in soup_txt:
            soup_txt.remove('')
        space=" "
        soup_txt=space.join(soup_txt)
        soup_txt=soup_txt.lower()
    #    if i%10 == 0:
    #        print(i)
        pattern=re.compile("item 2. manage")
        result_iter1=pattern.finditer(soup_txt)
        result_item2 = [ m1.span() for m1 in result_iter1]
        if len(result_item2) == 0:
            pattern=re.compile("item 2 – manage")
            result_iter1=pattern.finditer(soup_txt)
            result_item2 = [ m1.span() for m1 in result_iter1]
        if len(result_item2) == 0:
            pattern=re.compile("item 2.manage")
            result_iter1=pattern.finditer(soup_txt)
            result_item2 = [ m1.span() for m1 in result_iter1]
        pattern=re.compile("item 3. quant")
        result_iter1=pattern.finditer(soup_txt)
        result_item3 = [ m1.span() for m1 in result_iter1]
        pattern=re.compile("part ii")
        result_iter1=pattern.finditer(soup_txt)
        result_part2=[ m1.span() for m1 in result_iter1]
        print(len(result_item2))
        print(url)
        if len(result_item2) == 2:
            if len(result_item3) == 2:
                output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
            else:
                output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[1][0]]
        elif len(result_item2) == 1:
            output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_part2[0][0]]
        elif len(result_item2) > 3:
            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
        elif len(result_item2) == 3:
            output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
        else:
            print(len(result_item2))
            print(url)
            else_deal.append(i)
            print(i)
    except:
        print("unsolved")
        print(i)
"""

start
1
https://www.sec.gov/Archives/edgar/data/320193/000110465903009489/j0567_10q.htm
start


/shared_home/willie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1
https://www.sec.gov/Archives/edgar/data/320193/000110465904003080/a04-1622_110q.htm
start
1
https://www.sec.gov/Archives/edgar/data/320193/000110465904013021/a04-5089_110q.htm
start
1
https://www.sec.gov/Archives/edgar/data/320193/000110465905003520/a05-2329_110q.htm
start
1
https://www.sec.gov/Archives/edgar/data/320193/000110465905020421/a05-7551_110q.htm


In [89]:
def cut_item2(input_data):
    output=input_data.copy()
    output["text"]="NA"
    else_deal=[]
    for i in range(input_data.shape[0]):
        try:
            print("start")
            index_now=input_data.index[i]
            url="https://www.sec.gov"+input_data["url"].iloc[i]
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            soup_txt = soup.get_text()
            soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ')
            soup_txt=soup_txt.split(" ")
            while '' in soup_txt:
                soup_txt.remove('')
            space=" "
            soup_txt=space.join(soup_txt)
            soup_txt=soup_txt.lower()
        #    if i%10 == 0:
        #        print(i)
            pattern=re.compile("item 2. manage")
            result_iter1=pattern.finditer(soup_txt)
            result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2 – manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2.manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("item 3. quant")
            result_iter1=pattern.finditer(soup_txt)
            result_item3 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("part ii")
            result_iter1=pattern.finditer(soup_txt)
            result_part2=[ m1.span() for m1 in result_iter1]
            if len(result_item2) == 2:
                if len(result_item3) == 2:
                    output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
                else:
                    output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[1][0]]
            elif len(result_item2) == 1:
                output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_part2[0][0]]
            elif len(result_item2) > 3:
                output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
            elif len(result_item2) == 3:
                output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
            else:
                print(len(result_item2))
                print(url)
                else_deal.append(i)
                print(i)
        except:
            print("unsolved")
            print(i)
        return output

In [43]:
#FR_old = FR.copy()

In [ ]:
"""
#FR=pd.DataFrame(columns=["text"])
else_deal=[]
#for i in range(10):
for i in range(bad_date.shape[0]):
    try:
        if i not in FR.index:
            url="https://www.sec.gov"+bad_date["url"].iloc[i]
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            soup_txt = soup.get_text()
            soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ')
            soup_txt=soup_txt.split(" ")
            while '' in soup_txt:
                soup_txt.remove('')
            space=" "
            soup_txt=space.join(soup_txt)
            soup_txt=soup_txt.lower()
        #    if i%10 == 0:
        #        print(i)
            pattern=re.compile("item 2. manage")
            result_iter1=pattern.finditer(soup_txt)
            result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2 – manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            if len(result_item2) == 0:
                pattern=re.compile("item 2.manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("item 3. quant")
            result_iter1=pattern.finditer(soup_txt)
            result_item3 = [ m1.span() for m1 in result_iter1]
            pattern=re.compile("part ii")
            result_iter1=pattern.finditer(soup_txt)
            result_part2=[ m1.span() for m1 in result_iter1]
            if len(result_item2) == 2:
                if len(result_item3) == 2:
                    FR.loc[i]=soup_txt[result_item2[1][0]:result_item3[1][0]]
                else:
                    FR.loc[i]=soup_txt[result_item2[1][0]:result_part2[1][0]]
            elif len(result_item2) == 1:
                FR.loc[i]=soup_txt[result_item2[0][0]:result_part2[0][0]]
            elif len(result_item2) > 3:
                FR.loc[i]=soup_txt[result_item2[1][0]:result_item3[1][0]]
            elif len(result_item2) == 3:
                FR.loc[i]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
            else:
                print(len(result_item2))
                print(url)
                else_deal.append(i)
                print(i)
    except:
        print("unsolved")
        print(i)
"""

unsolved
75
unsolved
76
0
https://www.sec.gov/Archives/edgar/data/4611/0000950109-95-001847.txt
90
0
https://www.sec.gov/Archives/edgar/data/4611/0000950109-95-003188.txt
91
0
https://www.sec.gov/Archives/edgar/data/4611/0000004611-97-000012.txt
92
0
https://www.sec.gov/Archives/edgar/data/4611/000100547799003616/0001005477-99-003616.txt
93
unsolved
102
unsolved
103
unsolved
104
0
https://www.sec.gov/Archives/edgar/data/3333/0000891020-96-001074.txt
117
0
https://www.sec.gov/Archives/edgar/data/3333/000000333399000021/0000003333-99-000021.txt
118
0
https://www.sec.gov/Archives/edgar/data/3333/000000333300000030/0000003333-00-000030-0001.txt
119
0
https://www.sec.gov/Archives/edgar/data/719866/0000719866-98-000008.txt
134
0
https://www.sec.gov/Archives/edgar/data/883296/0000883296-97-000004.txt
194
unsolved
244
unsolved
245
unsolved
246
unsolved
260
unsolved
289
0
https://www.sec.gov/Archives/edgar/data/7084/000000708401500003/adm10q2qfy01.htm
297
unsolved
298
unsolved
299
0
https://www

unsolved
2158
0
https://www.sec.gov/Archives/edgar/data/15486/0000015486-95-000005.txt
2170
0
https://www.sec.gov/Archives/edgar/data/15486/0000015486-97-000003.txt
2171
0
https://www.sec.gov/Archives/edgar/data/15486/0000015486-98-000010.txt
2172
unsolved
2179
0
https://www.sec.gov/Archives/edgar/data/13610/0000950123-98-007659.txt
2189
0
https://www.sec.gov/Archives/edgar/data/13610/0000950123-99-004730.txt
2190
0
https://www.sec.gov/Archives/edgar/data/13610/000095012300005091/0000950123-00-005091-d1.html
2191
0
https://www.sec.gov/Archives/edgar/data/935226/0000935226-97-000007.txt
2193
0
https://www.sec.gov/Archives/edgar/data/46195/0000046195-98-000004.txt
2216
0
https://www.sec.gov/Archives/edgar/data/875357/000087535708000017/quarterly033108.txt
2219
0
https://www.sec.gov/Archives/edgar/data/875357/000087535709000017/quarterly033109.txt
2220
unsolved
2253
0
https://www.sec.gov/Archives/edgar/data/850033/0000950115-98-000978.txt
2269
0
https://www.sec.gov/Archives/edgar/data/850

0
https://www.sec.gov/Archives/edgar/data/928022/0000928022-98-000002.txt
3702
0
https://www.sec.gov/Archives/edgar/data/928022/000095013401501921/d87378e10-q.txt
3703
0
https://www.sec.gov/Archives/edgar/data/928022/000095013402005482/d96869e10-q.txt
3704
unsolved
3715
unsolved
3738
unsolved
3739
unsolved
3740
unsolved
3757
unsolved
3758
0
https://www.sec.gov/Archives/edgar/data/101063/000010106399000039/0000101063-99-000039.txt
3789
0
https://www.sec.gov/Archives/edgar/data/101063/000010106300000025/0000101063-00-000025-0001.txt
3790
0
https://www.sec.gov/Archives/edgar/data/101063/000119312508100201/d10q.htm
3791
unsolved
3793
unsolved
3794
unsolved
3796
0
https://www.sec.gov/Archives/edgar/data/778808/0000931763-98-001299.txt
3806
0
https://www.sec.gov/Archives/edgar/data/778808/000093176300000211/0000931763-00-000211.txt
3807
0
https://www.sec.gov/Archives/edgar/data/778808/000093176300001297/0000931763-00-001297.txt
3808
0
https://www.sec.gov/Archives/edgar/data/19161/0000019161-

unsolved
4924
unsolved
4927
unsolved
4928
0
https://www.sec.gov/Archives/edgar/data/27096/0000950123-96-000552.txt
4943
0
https://www.sec.gov/Archives/edgar/data/924940/000095013100004906/0000950131-00-004906-0001.txt
5003
unsolved
5017
0
https://www.sec.gov/Archives/edgar/data/27367/000110465900000480/0001104659-00-000480-0001.txt
5092
0
https://www.sec.gov/Archives/edgar/data/29332/0000029332-95-000007.txt
5096
0
https://www.sec.gov/Archives/edgar/data/29332/0000029332-98-000004.txt
5097
0
https://www.sec.gov/Archives/edgar/data/29332/0000029332-98-000006.txt
5098
0
https://www.sec.gov/Archives/edgar/data/907562/000091205702032396/a2086278z10-q.txt
5117
0
https://www.sec.gov/Archives/edgar/data/907562/000110465904012947/a04-5371_110q.htm
5118
0
https://www.sec.gov/Archives/edgar/data/907562/000110465904022299/a04-8656_110q.htm
5119
0
https://www.sec.gov/Archives/edgar/data/907562/000110465905036450/a05-12661_110q.htm
5120
0
https://www.sec.gov/Archives/edgar/data/907562/0001104659070

In [44]:
#FR['return'] = 0

In [45]:
#for i in range(FR.shape[0]):
#    FR["return"].iloc[i]=good_date_df["return"].iloc[FR.index[i]]


/shared_home/willie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [52]:
#FR_positive = FR[FR['text']!='']

In [32]:
#FR_positive = FR[FR["return"] != 0]

In [53]:
#FR_positive =FR_positive .dropna()
#FR_positive .to_csv("cutted_part_with_return_positive.csv")

In [ ]:
#for i in range(FR.shape[0]):
#    FR["return"].iloc[i]=bad_date["return"].iloc[FR.index[i]]

/shared_home/willie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/shared_home/willie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
#FR=pd.read_csv("cutted_part.csv",index_col=0)

In [30]:
#FR["return"]=0

In [116]:
#FR=FR.dropna()

In [120]:
#FR.to_csv("cutted_part_with_return.csv")

In [94]:
#FR.to_csv("cutted_part.csv")

In [90]:
"""
url="https://www.sec.gov"+bad_date["url"].iloc[76]
print(url)
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
soup_txt = soup.get_text()
soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ')
soup_txt=soup_txt.split(" ")
while '' in soup_txt:
    soup_txt.remove('')
space=" "
soup_txt=space.join(soup_txt)
soup_txt=soup_txt.lower()
#    if i%10 == 0:
#        print(i)
pattern=re.compile("item 2. manage")
result_iter1=pattern.finditer(soup_txt)
result_item2 = [ m1.span() for m1 in result_iter1]
if len(result_item2) == 0:
    pattern=re.compile("item 2 – manage")
    result_iter1=pattern.finditer(soup_txt)
    result_item2 = [ m1.span() for m1 in result_iter1]
if len(result_item2) == 0:
    pattern=re.compile("item 2.manage")
    result_iter1=pattern.finditer(soup_txt)
    result_item2 = [ m1.span() for m1 in result_iter1]
pattern=re.compile("item 3. quant")
result_iter1=pattern.finditer(soup_txt)
result_item3 = [ m1.span() for m1 in result_iter1]
pattern=re.compile("part ii")
result_iter1=pattern.finditer(soup_txt)
result_part2=[ m1.span() for m1 in result_iter1]
if len(result_item2) == 2:
    if len(result_item3) == 2:
        FR.loc[i]=soup_txt[result_item2[1][0]:result_item3[1][0]]
    else:
        FR.loc[i]=soup_txt[result_item2[1][0]:result_part2[1][0]]
elif len(result_item2) == 1:
    FR.loc[i]=soup_txt[result_item2[0][0]:result_part2[0][0]]
elif len(result_item2) > 3:
    FR.loc[i]=soup_txt[result_item2[1][0]:result_item3[1][0]]
elif len(result_item2) == 3:
    FR.loc[i]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
else:
    print(len(result_item2))
    print(url)
    else_deal.append(i)
    print(i)
"""

'\nurl="https://www.sec.gov"+bad_date["url"].iloc[76]\nprint(url)\nres = requests.get(url)\nsoup = BeautifulSoup(res.text, \'html.parser\')\nsoup_txt = soup.get_text()\nsoup_txt=soup_txt.replace(\'\xa0\',\' \').replace(\'&nbsp;\',\' \').replace(\'&#160;\',\' \').replace(\'\n\',\' \').replace(\'\x93\',\' \').replace(\'\x94\',\' \')\nsoup_txt=soup_txt.split(" ")\nwhile \'\' in soup_txt:\n    soup_txt.remove(\'\')\nspace=" "\nsoup_txt=space.join(soup_txt)\nsoup_txt=soup_txt.lower()\n#    if i%10 == 0:\n#        print(i)\npattern=re.compile("item 2. manage")\nresult_iter1=pattern.finditer(soup_txt)\nresult_item2 = [ m1.span() for m1 in result_iter1]\nif len(result_item2) == 0:\n    pattern=re.compile("item 2 – manage")\n    result_iter1=pattern.finditer(soup_txt)\n    result_item2 = [ m1.span() for m1 in result_iter1]\nif len(result_item2) == 0:\n    pattern=re.compile("item 2.manage")\n    result_iter1=pattern.finditer(soup_txt)\n    result_item2 = [ m1.span() for m1 in result_iter1]\npat

### Some test

In [349]:
import os

In [352]:
os.getcwd()

'/shared_home/willie/Fintech-Text-Mining-and-Machine-Learning-2019/HW2/sasa'

In [353]:
tickers = pd.read_csv('/shared_home/willie/Fintech-Text-Mining-and-Machine-Learning-2019/HW2/all_ticker.csv').iloc[:,1].values

In [368]:
dic_count2 = {}
dic_count3 = {}

for tk in tickers:
    dts = company_dt_dic[tk]
    fps = company_fp_dic[tk]
    if (dts == 'Unfounded'):
        dic_count2[tk] = 'Unfounded'
        dic_count3[tk] = 'Unfounded'
        continue
        
    for i in range(len(dts)):
        url="https://www.sec.gov"+fps[i]
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup_txt = soup.get_text()
        soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ').replace('\t',' ')
        soup_txt=soup_txt.split(" ")
        while '' in soup_txt:
            soup_txt.remove('')
        space=" "
        soup_txt=space.join(soup_txt)
        soup_txt=soup_txt.lower()
#         if soup_txt.count("item 2.")!=4:
#             break
        item2_count = soup_txt.count("item 2. manage")
        item3_count = soup_txt.count("item 3. quant")
        print(tk,i,item2_count,item3_count)
        dic_count2[tk] = item2_count
        dic_count3[tk] = item3_count
        

DSKE 0 2 2
DSKE 1 2 2
DSKE 2 2 2
DSKE 3 2 2
DSKE 4 2 2
DSKE 5 2 2
DSKE 6 2 2
DSKE 7 2 2
DSKE 8 2 2
DSKE 9 2 2
DSKE 10 2 2
CHIR 0 2 2
CHIR 1 2 2
CHIR 2 2 2
CHIR 3 3 3
CHIR 4 3 3


KeyboardInterrupt: 

In [369]:
url

'https://www.sec.gov/Archives/edgar/data/706539/000104746904014638/a2134460z10-q.htm'

In [364]:
space.join(soup_txt)

'10-Q 1 a2084731z10-q.htm 10-Q QuickLinks -- Click here to rapidly navigate through this document UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-Q (Mark one) ý QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934. For the quarterly period ended June 30, 2002 OR o TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934. For the transition period from to Commission File Number: 0-12798 CHIRON CORPORATION (Exact name of registrant as specified in its charter) Delaware 94-2754624 (State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.) 4560 Horton Street, Emeryville, California 94608 (Address of principal executive offices) (Zip code) (510) 655-8730 (Registrant\'s telephone number, including area code) Not Applicable (Former name, former address and former fiscal year, if changed since last report) Indicate by check mark whether the registrant (1) 

In [366]:
company_fp_dic['CHIR']

['/Archives/edgar/data/706539/000110465905052234/a05-17909_110q.htm',
 '/Archives/edgar/data/706539/000110465905036428/a05-12518_110q.htm',
 '/Archives/edgar/data/706539/000110465905021224/a05-7323_110q.htm',
 '/Archives/edgar/data/706539/000104746904032912/a2145286z10-q.htm',
 '/Archives/edgar/data/706539/000104746904025762/a2140097z10-q.htm',
 '/Archives/edgar/data/706539/000104746904014638/a2134460z10-q.htm',
 '/Archives/edgar/data/706539/000104746903037052/a2121800z10-q.htm',
 '/Archives/edgar/data/706539/000104746903027243/a2115051z10-q.htm',
 '/Archives/edgar/data/706539/000104746903016984/a2108383z10-q.htm',
 '/Archives/edgar/data/706539/000091205702041962/a2092069z10-q.htm',
 '/Archives/edgar/data/706539/000091205702030760/a2084731z10-q.htm',
 '/Archives/edgar/data/706539/000091205702019098/a2078994z10-q.htm',
 '/Archives/edgar/data/706539/000091205701538036/a2062292z10-q.htm',
 '/Archives/edgar/data/706539/000091205701527018/a2054786z10-q.htm',
 '/Archives/edgar/data/706539/00